In [1]:
from scipy.io import loadmat
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from tensorflow.keras.losses import Loss
from cal_error import ExpectedCalibrationError
import time 
from matplotlib import pyplot as plt

In [2]:
import sys
sys.path.insert(1, '/home/thlarsen/ood_detection')

from helper import load_mnist_c, load_mnist_model, rgb_img_to_vec
from cal_error import ExpectedCalibrationError


In [3]:
data, labels, sev = load_mnist_c('jpeg_compression')
step = 10000
mnist_by_sev = {}
for i in range(0, data.shape[0], step): 
    mnist_by_sev[sev[i]] = [data[i:i+step], labels[i:i+step]]

In [4]:
# A=0
# B=50000
# for i in range(A, A+B, 10000): 
#     print(f'label={labels[i]}, sev={sev[i]}')
#     plt.imshow(data[i])
#     plt.show()
    

In [5]:
model = load_mnist_model() 

In [6]:
acc_fn = keras.metrics.SparseCategoricalAccuracy()
ece = ExpectedCalibrationError()

In [7]:
for sev in mnist_by_sev.keys(): 
    data_s, labels_s = mnist_by_sev[sev]
    print(data_s.shape)
    data_s = rgb_img_to_vec(data_s)

    preds = model.predict(np.reshape(data_s, (-1, 1024)))
    acc_fn = keras.metrics.Accuracy()
    print(f" acc = {acc_fn(labels_s, preds)}")
    ece = ExpectedCalibrationError()
    print(f" ece = {ece.call(labels_s, preds)}")


(10000, 32, 32, 3)


ValueError: Shapes (10000,) and (10000, 10) are incompatible